# Daten Erkundung: Exercise

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import plotly.express as px
from statsmodels.tsa.stattools import adfuller

In [ ]:
!curl -L https://github.com/NikolausHouben/HAB_Strom/blob/d0ac42bb653de446ab88d503a90433519dff6bfb/Daten/data_1.csv\?raw\=true -o data_1.csv
df_1 = pd.read_csv("data_1.csv", index_col = 0, parse_dates = True).resample("60T").mean()

!curl -L https://github.com/NikolausHouben/HAB_Strom/blob/d0ac42bb653de446ab88d503a90433519dff6bfb/Daten/data_2.csv\?raw\=true -o data_2.csv
df_2 = pd.read_csv("data_2.csv", index_col = 0, parse_dates = True).resample("60T").mean()



## Konzept 1: Stationarität

Stationarität ist eine der bedeutendsten Eigenschaften stochastischer Prozesse in der Zeitreihenanalyse. 

Mit der Stationarität erhält man Eigenschaften, die nicht nur für einzelne Zeitpunkte gelten, sondern Invarianzen über die Zeit hinweg sind. 
Die Zeitreihe hat zu allen Zeitpunkten den gleichen Erwartungswert und die gleiche Varianz.


https://de.wikipedia.org/wiki/Stationärer_stochastischer_Prozess


Es gibt drei Bedingungen damit ein stochastischer Prozess (schwach) stationär ist:

* Der Erwartungswert ist konstant über die Zeitreihe
* Die Varianz ist endlich
* Die Autokovarianz: Kovarianzen zwischen den Zeitpunkten sind nicht von deren Verschiebung abhängig, sondern nur vom Abstand




Eine nichtstationäre Zeitreihe stationär zu machen ist eine wichtige erste Aufgabe bei der Zeitreihenanalyse. Weit verbreitete Methoden sind hier die Bildung von Differenzen, das Umskalieren oder das Logarithmieren der Zeitreihe.






*Ist meine Zeitreihe stationär?*

* Um festzustellen ob eine Reihe stationär ist, können wir den erweiterten Dickey-Fuller-Test verwenden. In diesem Test besagt die Nullhypothese, dass
$\phi = 1$ 
   (dies wird auch als Unit-Test bezeichnet). Die Null-Hypothese besagt, dass unsere Zeitreihe nicht stationär ist.

* Der Test gibt mehrere Statistiken zurück, die wir gleich sehen werden. Unser Fokus liegt auf dem p-Wert. Ein kleiner p-Wert ($p<0.05$) weist auf starke Evidenz gegen die Nullhypothese hin.


WICHTIG: Stationarität spielt innerhalb des "Forecast Horizon" eine besondere Rolle. Eine Zeitreihe kann als ganzes stationär sein, aber innerhalb des Horizonts nicht mehr. Das kann zu Schwierigkeiten in der Modellierung führen (siehe Slides – p. XX )


In [ ]:
def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")






In [ ]:
forecast_horizon = 24 #hours
adf_test(df_2[:(df_2.index[0] + pd.Timedelta(hours = forecast_horizon))])
df_2[:(df_2.index[0] + pd.Timedelta(hours = forecast_horizon))].plot()

In [ ]:
adf_test(df_1[:(df_1.index[0] + pd.Timedelta(hours = forecast_horizon))])
df_1[:(df_1.index[0] + pd.Timedelta(hours = forecast_horizon))].plot()

## Konzept 2: Autokorrelation

### Autokovarianz


"Auto" heißt mit sich selbst, dh. Kovarianz nicht zwischen zwei unterschiedlichen Zeitreihen sondern eine Zeitreihe mit sich selbst aber "lagged".


Bei einer stationären Zeitreihe, ist die Auto-Kovarianz Funktion $\gamma$:

${\displaystyle {\gamma}_{XX}(t_{1},t_{2})=\operatorname {Cov} \left[X_{t_{1}},X_{t_{2}}\right]=\operatorname {E} [(X_{t_{1}}-\mu _{t_{1}})(X_{t_{2}}-\mu _{t_{2}})]}$

Ein ausgewähltes $\gamma_k$ können wir dann wie folgt berechnen:

${\displaystyle \gamma_k = \frac 1 n \sum\limits_{t=1}^{n-k} (y_t - \bar{y})(y_{t+k}-\bar{y})}$

### Die Auto-Korrelation $\rho_k$ von einer Zeitreihe bei "Lag" $k$ ist:

${\displaystyle \rho_k = \frac {\sum\limits_{t=1}^{n-k} (y_t - \bar{y})(y_{t+k}-\bar{y})} {\sum\limits_{t=1}^{n} (y_t - \bar{y})^2}}$

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
title = 'Auto-Korrelation: df_1'
lags = 4*24 #in timesteps
plot_acf(df_1,title=title,lags=lags);

In [ ]:
title = 'Auto-Korrelation: df_2'
lags = 4*24
plot_acf(df_2,title=title,lags=lags);

## Konzepte 3, 4: Periodizität & Rauschen

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

df_1_series = df_1["y_1"].resample("15T").mean().fillna(method = "ffill") #damit df_1 lesbar für "seasonal_decompose" wird.

df_1_series = df_1_series[:df_1_series.index[0] + pd.Timedelta(weeks = 5)]
result = seasonal_decompose(df_1_series, model='additive', period= 4*24)


In [ ]:
px.line(result.seasonal)

In [ ]:
px.line(result.resid)